救急データをくっつけてクリーンするだけ  
outputに保存される  
初回のみ、もしくは作業済みのnumpyファイルがないときのみ  

In [1]:
import os
import sys

sys.path.append(os.path.join(os.path.abspath(''), '../src/'))
import pandas as pd

def clean_data(df):
    # 次の列の種類をstringに変更する: '覚知日付(元号年月日)'
    df = df.astype({'覚知日付(元号年月日)': 'string'})
    df = df.astype({'覚知日付(元号)名': 'string'})
    df = df.astype({'覚知日付(年)': 'string'})
    df = df.astype({'覚知日付(月)': 'string'})
    df = df.astype({'覚知日付(日)': 'string'})
    df = df.astype({'覚知時刻(時)': 'string'})
    df = df.astype({'覚知時刻(分)': 'string'})

    df["datestr"] = df["覚知日付(元号)名"] + df["覚知日付(年)"] +"年" + df["覚知日付(月)"] +"月"+ df["覚知日付(日)"]+"日" \
                    + " " + df["覚知時刻(時)"] + ":" + df["覚知時刻(分)"]
    df.drop(['覚知日付(元号年月日)','覚知日付(元号)名','覚知日付(年)',
             '覚知日付(月)','覚知日付(日)','覚知時刻(時)','覚知時刻(分)'],axis=1,inplace=True)
    import jp_datetime
    df["datetime"] = df["datestr"].apply(lambda x: 
                                         jp_datetime.parse_date(x,'%g%e年%m月%d日 %H:%M')
                                         )
    df.drop(['datestr'],axis=1,inplace=True)
    df = df.reindex(['datetime','出場隊名', '事故種別(基本)名', '覚知→現着(所要時間)', '覚知→病着(所要時間)', '年齢(歳)', '年齢区分名',
       '性別名', '傷病程度名', '傷病名', '傷病名大分類(国)名', '傷病名中分類(国)名', '傷病名小分類(国)名',
       '収容機関名', '行政区別区分名'], axis=1)
    

    # 病院に実際に来る場合を対象にしたいので、不搬送を除去
    df = df[df['収容機関名'] != "不搬送"]

    # 列を削除: '傷病名大分類(国)名'
    df = df.drop(columns=['傷病名大分類(国)名'])
    df = df.drop(columns=['傷病名中分類(国)名'])
    df = df.drop(columns=['傷病名小分類(国)名'])

    # count用に列を追加
    df["count"] = 1

    # 列で並べ替えます: 'datetime' (昇順)
    df = df.sort_values(['datetime'], ignore_index=True)
    
    return df

# excel形式のデータ読み込み
df = pd.concat(
    [pd.read_excel(r'../input/01_令和元年.xlsx'),
    pd.read_excel(r"../input/02_令和２年.xlsx"),
    pd.read_excel(r"../input/03_令和３年.xlsx"),
    pd.read_excel(r"../input/04_令和４年.xlsx"),
    pd.read_excel(r"../input/05_令和５年.xlsx")]
)
df_clean = clean_data(df.copy())
df_clean.head(3)
# df_clean.傷病名.to_csv("../output/傷病名一覧.csv",index=None)
# 保存
df_clean.to_pickle('../output/df_clean.pkl.gz', compression='gzip') # 圧縮有り